In [1]:
# Dependencies
import requests
import os
from os.path  import basename
from bs4 import BeautifulSoup
import pandas as pd
from glob import glob
import datetime
from datetime import datetime
import time
import copy
from random import randint
import unicodedata
import pprint
import json

In [2]:
leagues_df = pd.read_csv("../../data/leagues_data.csv", index_col=False)

In [3]:
soccer_data = {"Leagues": leagues_df.to_dict(orient='records')}
pprint.pprint(soccer_data)

{'Leagues': [{'country': 'Brazil',
              'league_ID': 1,
              'league_link': 'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1',
              'league_name': 'Brazil Serie A',
              'tier': 1},
             {'country': 'United States',
              'league_ID': 2,
              'league_link': 'https://www.transfermarkt.com/major-league-soccer/startseite/wettbewerb/MLS1',
              'league_name': 'MLS (US)',
              'tier': 1},
             {'country': 'Mexico',
              'league_ID': 3,
              'league_link': 'https://www.transfermarkt.com/liga-mx-clausura/startseite/wettbewerb/MEX1',
              'league_name': 'Liga MX (MEX)',
              'tier': 1},
             {'country': 'Argentina',
              'league_ID': 4,
              'league_link': 'https://www.transfermarkt.com/primera-division/startseite/wettbewerb/AR1N',
              'league_name': 'Superliga (ARG)',
              'tier': 1}]}


In [4]:
# to be able to scrape this website we need to use 'User Agents'
# more info about user agents in 'https://webscraping.com/blog/User-agents/'
# You can find your User-Agent at 'http://whatsmyuseragent.com/'
headers = {'User-Agent': 
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'}

In [5]:
convert_url = "https://www.google.com/search?q=euro+to+dollar&oq=eur&aqs=chrome.1.69i57j35i39j0j46j0l2j69i61l2.2373j1j4&sourceid=chrome&ie=UTF-8"
html = requests.get(convert_url, headers=headers)
soup = BeautifulSoup(html.content, 'html.parser')
curr_value = soup.find('div', class_= "b1hJbf")
curr_value = round(float(curr_value["data-exchange-rate"]),2)

def get_value_us(x):
    value = []
    for char in x:
        value.append(char)
    if value[-1] == "m":
        float_value = "".join(value[1:-1])
        return round(float(float_value)*curr_value,2)
    else:
        float_value = "".join(value[1:-3])
        return round((float(float_value)*curr_value)/1000,2)


In [6]:
def strip_accents(text):

    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return str(text.strip())

In [7]:
def fix_heights(soccer_data1):
    for player in soccer_data1['Players']:
        
        if player["Height"] == 0:
            list_height= []
            targ_player = player
            nat = targ_player["Nat"]
            position = targ_player["position"]
            field_position = targ_player["field_position"]
#             print("Targuet Player",targ_player["name"])
            for player in soccer_data1['Players']:
                if player["position"] == position and player["Nat"] == nat:
                    if player["Height"] != 0:
#                         print(player["name"])
                        list_height.append(player["Height"])

                if len(list_height) == 0:
#                     print("lista vazia1", targ_player["name"])
                    if player["field_position"] == field_position and player["Nat"] == nat:
                        if player["Height"] != 0:
#                             print(player["name"])
                            list_height.append(player["Height"])

                if len(list_height) == 0:
#                     print("lista vazia2", targ_player["name"])
                    if player["field_position"] == field_position:
                        if player["Height"] != 0:
#                             print(player["name"])
                            list_height.append(player["Height"])

#             print("----antes",targ_player["Height"], targ_player["name"])
            targ_player["Height"] = round(sum(list_height) / len(list_height),2)
#             print(list_height)
#             print("--- depois", targ_player["name"], targ_player["Height"])


In [8]:
def scrape_league_data(soccer_data):
    team_id = 1
    
    for league in soccer_data["Leagues"]:
        teams = []
        leaguename = league['league_name']
        url = league["league_link"]
        print(f"scraping: {league['country']}_{league['tier']}_{league['league_name']}")
        time.sleep(3)
        html = requests.get(url, headers=headers)
        soup = BeautifulSoup(html.content, 'html.parser')
        htmltable = soup.find('table', class_= "items")
        results = htmltable.findAll("tr", class_ =["odd","even"])
#         league['clubs'] = {}
        for result in results:
            features = result.findAll("td")
            link = ("https://www.transfermarkt.com"+result.find("a", href=True)\
                          ["href"]+"/plus/1").replace("startseite", "kader")
            logo = result.find("img", src=True)["src"]
            logo = logo.split("?")[0]
            logo = logo.replace("tiny", "header")
            name = strip_accents(features[1].text)
            squad = int(features[3].text)
            foreigner = int(features[5].text)
            total_MV = float(get_value_us(features[6].text))
            avg_MV = float(get_value_us(features[7].text))
            
#             # All data
            teams.append({"team_ID" : team_id, "league_ID":league["league_ID"], 'league_name': leaguename,
                          "club" : name, "squad" : squad, "foreigners" : foreigner, "avg_market_m" : avg_MV,
                             "total_MV_m" : total_MV, "Logo_img" : logo, "link_page" : link})
              
#             # hierachy
#             teams.append({"team_ID" : team_id,"club" : name, "squad" : squad, "foreigners" : foreigner, "avg_market_m" : avg_MV,
#                           "total_MV_m" : total_MV, "Logo_img" : logo, "link_page" : link})
            team_id = team_id + 1
        league["Clubs"] = teams
#     soccer_data['Clubs'] = teams




In [9]:
def scrape_team_data(soccer_data):
    player_id = 1
    players = []
    for league in soccer_data["Leagues"]:
        for club in league["Clubs"]:
            team_id = club["team_ID"]
            team_link = club["link_page"]
            league_id = club["league_ID"]
            team_name = club["club"]
            print(club["club"])
            time.sleep(randint(1,2))
            html = requests.get(team_link, headers=headers)
            soup = BeautifulSoup(html.content, 'html.parser')
            htmltable = soup.find('table', class_= "items")
            results = htmltable.findAll("tr", class_ =["odd","even"])

            for result in results:
                features = result.findAll("td")

                name_1 = features[2].find("img", alt=True)["alt"]
                name_1 = strip_accents(name_1)

                player_page = "https://www.transfermarkt.com" + features[3].find("a", href=True)["href"]

                position_1 = features[4].text

                if position_1 in ("Centre-Back", "Left-Back", "Right-Back", "Defender"):
                    field_posit = "DEF"
                elif position_1 in ("Defensive Midfield", "Central Midfield" , "Right Midfield" , "Left Midfield" , \
                "Attacking Midfield" , "Midfield"):
                    field_posit = "MID"
                elif position_1 in ("Left Winger" , "Right Winger" , "Centre-Forward" , "Second Striker" , "Forward"):
                    field_posit = "ATT"
                else:
                    field_posit = "GLK"

                try:
                    age_1 = int((features[5].text.split("(",)[-1])[:-1])
                except:
                    age_1 = ""

                nat = features[6].img["alt"]

                if league_id == 3 or   league_id == 4:
                    try:
                        Height_1 = float((features[8].text.split(" ")[0]).replace(",", "."))
                    except:
                        Height_1 = ""

                    foot_1 = features[9].text
                    dt_joined_1 = features[10].text
                    try:
                        dt_joined_1 = datetime.strptime(dt_joined_1, '%b %d, %Y').date()
                    except:
                        dt_joined_1 = ""
                    try:
                        prev_team_1 = features[11].img["alt"]
                    except:
                        prev_team_1 = "N.A."

                    contract_expires_1 = features[12].text
                    try:
                        contract_expires_1 = datetime.strptime(contract_expires_1, '%d.%m.%Y').date()
                    except:
                        contract_expires_1 = ""

                    try:
                        market_value_1 = get_value_us(features[13].text[:-2])
                    except:
                        market_value_1 = 0
                else:
                    try:
                        Height_1 = float((features[7].text.split(" ")[0]).replace(",", "."))
                    except:
                        Height_1 = 0

                    foot_1 = features[8].text

                    dt_joined_1 = features[9].text
                    try:
                        dt_joined_1 = datetime.strptime(dt_joined_1, '%b %d, %Y').date()
                    except:
                        dt_joined_1 = ""

                    try:
                        prev_team_1 = features[10].img["alt"]
                    except:
                        prev_team_1 = "N.A."


                    contract_expires_1 = features[11].text
                    try:
                        contract_expires_1 = datetime.strptime(contract_expires_1, '%d.%m.%Y').date()
                    except:
                        contract_expires_1 = ""

                    try:
                        market_value_1 = get_value_us(features[12].text[:-2])
                    except:
                        market_value_1 = 0

                players.append({"players_ID": player_id, "team_ID": team_id, "club": team_name,
                                "Logo_img":club["Logo_img"], "league_ID":club["league_ID"], "league_name": club["league_name"], 
                                "name": name_1, "position": position_1,"field_position": field_posit, "Age":age_1,"Nat": nat, 
                                "Height": Height_1, "foot": foot_1,  'dt_joined': dt_joined_1,"prev_team": prev_team_1,
                                "contract_expires": contract_expires_1, "market_value": market_value_1,"player_page": player_page}) 
                player_id = player_id+1
        # players dentro dos clubes (hierarchy)
#         club["Players"] = players

# players separado dos clubs #
    soccer_data["Players"] = players 
        

In [10]:
scrape_league_data(soccer_data)

scraping: Brazil_1_Brazil Serie A
scraping: United States_1_MLS (US)
scraping: Mexico_1_Liga MX (MEX)
scraping: Argentina_1_Superliga (ARG)


In [11]:
scrape_team_data(soccer_data)

Clube de Regatas do Flamengo
Sociedade Esportiva Palmeiras
Gremio Foot-Ball Porto Alegrense
Sport Club Corinthians Paulista
Sport Club Internacional
Clube Atletico Mineiro
Sao Paulo Futebol Clube
Santos FC
Fluminense Football Club
Club Athletico Paranaense
Club de Regatas Vasco da Gama
Red Bull Bragantino
Esporte Clube Bahia
Botafogo Rio de Janeiro
Goias Esporte Clube
Coritiba Foot Ball Club
Fortaleza Esporte Clube
Ceara Sporting Club
Sport Club do Recife
Atletico Clube Goianiense
Los Angeles FC
Inter Miami CF
Atlanta United FC
New York City FC
Toronto FC
Los Angeles Galaxy
Seattle Sounders FC
Columbus Crew SC
Sporting Kansas City
FC Dallas
Portland Timbers
Minnesota United FC
D.C. United
FC Cincinnati
Chicago Fire FC
Orlando City SC
Colorado Rapids
New England Revolution
Houston Dynamo
New York Red Bulls
Philadelphia Union
Montreal Impact
Vancouver Whitecaps FC
Nashville SC
San Jose Earthquakes
Real Salt Lake City
CF Monterrey
CF America
Tigres UANL
CD Cruz Azul
CF Pachuca
Deportivo G

In [12]:
# with open('testdata3.json', 'w') as fp:
#     json.dump(soccer_data, fp)

In [13]:
fix_heights(soccer_data)
with open('dict_soccer_data.json', 'w') as fp:
#     json.dump(soccer_data, fp, indent=4, sort_keys=True, default=str)
    json.dump(soccer_data, fp, default=str)    

In [14]:
# pprint.pprint(soccer_data["Players"])

[{'Age': 35,
  'Height': 1.87,
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/614.png',
  'Nat': 'Brazil',
  'club': 'Clube de Regatas do Flamengo',
  'contract_expires': datetime.date(2020, 12, 31),
  'dt_joined': datetime.date(2017, 7, 16),
  'field_position': 'GLK',
  'foot': 'left',
  'league_ID': 1,
  'league_name': 'Brazil Serie A',
  'market_value': 3.3,
  'name': 'Diego Alves',
  'player_page': 'https://www.transfermarkt.com/diego-alves/profil/spieler/42373',
  'players_ID': 1,
  'position': 'Goalkeeper',
  'prev_team': 'Valencia CF',
  'team_ID': 1},
 {'Age': 21,
  'Height': 1.96,
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/614.png',
  'Nat': 'Brazil',
  'club': 'Clube de Regatas do Flamengo',
  'contract_expires': datetime.date(2023, 9, 30),
  'dt_joined': datetime.date(2019, 8, 31),
  'field_position': 'GLK',
  'foot': 'both',
  'league_ID': 1,
  'league_name': 'Brazil Serie A',
  'market_value': 0.53,
  'name': 'Hugo Souza',
  'player_pa

  'contract_expires': datetime.date(2021, 12, 31),
  'dt_joined': datetime.date(2017, 1, 11),
  'field_position': 'DEF',
  'foot': 'right',
  'league_ID': 1,
  'league_name': 'Brazil Serie A',
  'market_value': 1.18,
  'name': 'Fabiano',
  'player_page': 'https://www.transfermarkt.com/fabiano/profil/spieler/271335',
  'players_ID': 42,
  'position': 'Right-Back',
  'prev_team': 'Cruzeiro Esporte Clube',
  'team_ID': 2},
 {'Age': 19,
  'Height': 1.76,
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/1023.png',
  'Nat': 'Brazil',
  'club': 'Sociedade Esportiva Palmeiras',
  'contract_expires': datetime.date(2024, 12, 31),
  'dt_joined': datetime.date(2020, 1, 1),
  'field_position': 'MID',
  'foot': 'right',
  'league_ID': 1,
  'league_name': 'Brazil Serie A',
  'market_value': 4.25,
  'name': 'Gabriel Menino',
  'player_page': 'https://www.transfermarkt.com/gabriel-menino/profil/spieler/608216',
  'players_ID': 43,
  'position': 'Defensive Midfield',
  'prev_team': 'SE Pa

  'contract_expires': datetime.date(2021, 6, 30),
  'dt_joined': datetime.date(2020, 1, 1),
  'field_position': 'ATT',
  'foot': 'right',
  'league_ID': 1,
  'league_name': 'Brazil Serie A',
  'market_value': 0.94,
  'name': 'Ferreira',
  'player_page': 'https://www.transfermarkt.com/ferreira/profil/spieler/689017',
  'players_ID': 92,
  'position': 'Left Winger',
  'prev_team': 'Grêmio Foot-Ball Porto Alegrense B',
  'team_ID': 3},
 {'Age': 27,
  'Height': 1.74,
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/210.png',
  'Nat': 'Brazil',
  'club': 'Gremio Foot-Ball Porto Alegrense',
  'contract_expires': datetime.date(2023, 12, 31),
  'dt_joined': datetime.date(2018, 1, 2),
  'field_position': 'ATT',
  'foot': 'right',
  'league_ID': 1,
  'league_name': 'Brazil Serie A',
  'market_value': 3.78,
  'name': 'Alisson',
  'player_page': 'https://www.transfermarkt.com/alisson/profil/spieler/229736',
  'players_ID': 93,
  'position': 'Right Winger',
  'prev_team': 'Cruzeiro E

  'team_ID': 4},
 {'Age': 35,
  'Height': 1.85,
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/199.png',
  'Nat': 'Argentina',
  'club': 'Sport Club Corinthians Paulista',
  'contract_expires': datetime.date(2020, 12, 30),
  'dt_joined': datetime.date(2019, 1, 3),
  'field_position': 'ATT',
  'foot': 'right',
  'league_ID': 1,
  'league_name': 'Brazil Serie A',
  'market_value': 1.42,
  'name': 'Mauro Boselli',
  'player_page': 'https://www.transfermarkt.com/mauro-boselli/profil/spieler/40822',
  'players_ID': 129,
  'position': 'Centre-Forward',
  'prev_team': 'Club León FC',
  'team_ID': 4},
 {'Age': 33,
  'Height': 1.88,
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/6600.png',
  'Nat': 'Brazil',
  'club': 'Sport Club Internacional',
  'contract_expires': datetime.date(2021, 12, 31),
  'dt_joined': datetime.date(2016, 7, 15),
  'field_position': 'GLK',
  'foot': 'right',
  'league_ID': 1,
  'league_name': 'Brazil Serie A',
  'market_value': 1.65,
  

  'position': 'Goalkeeper',
  'prev_team': 'Cruzeiro Esporte Clube',
  'team_ID': 6},
 {'Age': 21,
  'Height': 1.9,
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/330.png',
  'Nat': 'Brazil',
  'club': 'Clube Atletico Mineiro',
  'contract_expires': datetime.date(2021, 12, 31),
  'dt_joined': datetime.date(2020, 1, 1),
  'field_position': 'GLK',
  'foot': '-',
  'league_ID': 1,
  'league_name': 'Brazil Serie A',
  'market_value': 0,
  'name': 'Matheus Mendes',
  'player_page': 'https://www.transfermarkt.com/matheus-mendes/profil/spieler/629077',
  'players_ID': 169,
  'position': 'Goalkeeper',
  'prev_team': 'Clube Atlético Mineiro B',
  'team_ID': 6},
 {'Age': 27,
  'Height': 1.84,
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/330.png',
  'Nat': 'Paraguay',
  'club': 'Clube Atletico Mineiro',
  'contract_expires': datetime.date(2024, 6, 30),
  'dt_joined': datetime.date(2020, 7, 2),
  'field_position': 'DEF',
  'foot': 'left',
  'league_ID': 1,
  'le

...

  'Nat': 'Bolivia',
  'club': 'Club Atletico Tucuman',
  'contract_expires': '',
  'dt_joined': '',
  'field_position': 'MID',
  'foot': 'right',
  'league_ID': 4,
  'league_name': 'Superliga (ARG)',
  'market_value': 0.56,
  'name': 'Raul Castro',
  'player_page': 'https://www.transfermarkt.com/raul-castro/profil/spieler/320654',
  'players_ID': 2818,
  'position': 'Central Midfield',
  'prev_team': 'N.A.',
  'team_ID': 84},
 {'Age': 33,
  'Height': 1.76,
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/14554.png',
  'Nat': 'Argentina',
  'club': 'Club Atletico Tucuman',
  'contract_expires': datetime.date(2020, 6, 30),
  'dt_joined': datetime.date(2019, 7, 22),
  'field_position': 'MID',
  'foot': 'left',
  'league_ID': 4,
  'league_name': 'Superliga (ARG)',
  'market_value': 0.41,
  'name': 'Ariel Rojas',
  'player_page': 'https://www.transfermarkt.com/ariel-rojas/profil/spieler/57753',
  'players_ID': 2819,
  'position': 'Central Midfield',
  'prev_team': 'Club Atlét

  'league_ID': 4,
  'league_name': 'Superliga (ARG)',
  'market_value': 0.29,
  'name': 'Nahuel Yeri',
  'player_page': 'https://www.transfermarkt.com/nahuel-yeri/profil/spieler/140603',
  'players_ID': 2849,
  'position': 'Right-Back',
  'prev_team': 'Club Atlético Banfield II',
  'team_ID': 85},
 {'Age': 25,
  'Height': 1.84,
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/12301.png',
  'Nat': 'Argentina',
  'club': 'Club Atletico Aldosivi',
  'contract_expires': datetime.date(2020, 6, 30),
  'dt_joined': datetime.date(2019, 7, 1),
  'field_position': 'DEF',
  'foot': 'right',
  'league_ID': 4,
  'league_name': 'Superliga (ARG)',
  'market_value': 0.12,
  'name': 'Cain Fara',
  'player_page': 'https://www.transfermarkt.com/cain-fara/profil/spieler/619072',
  'players_ID': 2850,
  'position': 'Right-Back',
  'prev_team': 'Club Ferro Carril Oeste',
  'team_ID': 85},
 {'Age': 26,
  'Height': 1.77,
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/12301.png'

  'league_ID': 4,
  'league_name': 'Superliga (ARG)',
  'market_value': 0.12,
  'name': 'Mateo Montenegro',
  'player_page': 'https://www.transfermarkt.com/mateo-montenegro/profil/spieler/728287',
  'players_ID': 2889,
  'position': 'Defensive Midfield',
  'prev_team': 'Club Atlético Central Córdoba (SdE) II',
  'team_ID': 86},
 {'Age': 27,
  'Height': 1.77,
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/31284.png',
  'Nat': 'Argentina',
  'club': 'Club Atletico Central Cordoba (SdE)',
  'contract_expires': datetime.date(2020, 6, 30),
  'dt_joined': datetime.date(2019, 7, 1),
  'field_position': 'MID',
  'foot': 'right',
  'league_ID': 4,
  'league_name': 'Superliga (ARG)',
  'market_value': 1.12,
  'name': 'Marcelo Meli',
  'player_page': 'https://www.transfermarkt.com/marcelo-meli/profil/spieler/263557',
  'players_ID': 2890,
  'position': 'Central Midfield',
  'prev_team': 'Racing Club',
  'team_ID': 86},
 {'Age': 22,
  'Height': 1.75,
  'Logo_img': 'https://tmssl.a

  'contract_expires': datetime.date(2021, 12, 31),
  'dt_joined': datetime.date(2016, 1, 6),
  'field_position': 'MID',
  'foot': 'right',
  'league_ID': 4,
  'league_name': 'Superliga (ARG)',
  'market_value': 0.29,
  'name': 'Damian Lemos',
  'player_page': 'https://www.transfermarkt.com/damian-lemos/profil/spieler/68557',
  'players_ID': 2923,
  'position': 'Defensive Midfield',
  'prev_team': 'Nueva Chicago',
  'team_ID': 87},
 {'Age': 22,
  'Height': 1.78,
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/19806.png',
  'Nat': 'Argentina',
  'club': 'Club Atletico Patronato',
  'contract_expires': datetime.date(2021, 6, 30),
  'dt_joined': datetime.date(2020, 1, 1),
  'field_position': 'MID',
  'foot': 'right',
  'league_ID': 4,
  'league_name': 'Superliga (ARG)',
  'market_value': 0.35,
  'name': 'Lautaro Torres',
  'player_page': 'https://www.transfermarkt.com/lautaro-torres/profil/spieler/486237',
  'players_ID': 2924,
  'position': 'Central Midfield',
  'prev_team

 {'Age': 32,
  'Height': '',
  'Logo_img': 'https://tmssl.akamaized.net/images/wappen/header/4673.png',
  'Nat': 'Argentina',
  'club': 'Arsenal Futbol Club',
  'contract_expires': datetime.date(2020, 6, 30),
  'dt_joined': datetime.date(2018, 7, 4),
  'field_position': 'ATT',
  'foot': '-',
  'league_ID': 4,
  'league_name': 'Superliga (ARG)',
  'market_value': 0.18,
  'name': 'Ezequiel Cerica',
  'player_page': 'https://www.transfermarkt.com/ezequiel-cerica/profil/spieler/440181',
  'players_ID': 2969,
  'position': 'Centre-Forward',
  'prev_team': 'Club Atlético Los Andes',
  'team_ID': 88}]


# TESTES

In [15]:
# with open('testdata5.json') as f:
#     soccer_data1 = json.load(f)



In [16]:
# for league in soccer_data1["Leagues"]:
#     clubs = []
# #     clubs = 1
#     for club in soccer_data1["Clubs"]:
# #     for league in soccer_data1["Leagues"]:
# #         clubs = 1
#         if club["league_ID"] == league["league_ID"]:
# #             print(club["club"], league["league_name"], clubs)
# #             clubs = clubs +1
# #     print(clubs, "-----")
            
#             clubs.append(club)
#     league["Clubs"] = clubs
    
# del soccer_data1["Clubs"]

In [17]:
# with open('testdata5.json', 'w') as fp:
#     json.dump(soccer_data1, fp)

## Save scraped data on SQL Database

In [18]:
# from sqlalchemy import *
# from sqlalchemy.schema import *
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy.sql import text
# import contextlib 
# import json

In [19]:
# rds_connection_string = "postgres:PostgreSQL@localhost:5432/trfmk_scrape_db"
# engine = create_engine(f'postgresql://{rds_connection_string}')
# metadata = MetaData()


In [20]:
# leagues_df = pd.read_csv("data/leagues_data.csv")
# teams_df = pd.read_csv("data/teams_trmk.csv")
# # fix 
# players_df = pd.read_csv("data/players_trmk1.csv")

In [21]:
# engine.execute(DropSchema('public', cascade = True))
# engine.execute('CREATE SCHEMA IF NOT EXISTS public;')

# players_df.to_sql(name = 'players',index_label= None, index= False, con=engine, if_exists='replace')
# teams_df.to_sql(name = 'teams', index_label= None, index= False, con=engine, if_exists='replace')
# leagues_df.to_sql(name = 'leagues', index_label= None, index= False, con=engine, if_exists='replace')

# with engine.connect() as con:
#     con.execute('ALTER TABLE leagues ADD CONSTRAINT pk_Leagues PRIMARY KEY ("league_ID")')
#     con.execute('ALTER TABLE teams ADD CONSTRAINT pk_Teams PRIMARY KEY ("team_ID")')
#     con.execute('ALTER TABLE players ADD CONSTRAINT pk_Players PRIMARY KEY ("players_ID")')
#     con.execute('ALTER TABLE Teams ADD CONSTRAINT fk_Teams_league_ID FOREIGN KEY("league_ID") REFERENCES Leagues ("league_ID");')
#     con.execute('ALTER TABLE Players ADD CONSTRAINT fk_Players_team_ID FOREIGN KEY("team_ID") REFERENCES Teams ("team_ID");')

In [22]:
# # read database
# pd.read_sql_query('SELECT * FROM  leagues JOIN (teams JOIN players USING("team_ID")) USING ("league_ID")', con=engine)

### Save query as JSON file

In [23]:
# pd.read_sql_query('SELECT * FROM  leagues JOIN (teams JOIN players USING("team_ID")) USING ("league_ID")', con=engine)\
# .to_json(r'dashboard/static/data/data-dict.json', orient='records')

In [24]:
# print(json.dumps(pd.read_sql_query('SELECT * FROM  leagues JOIN (teams JOIN players USING("team_ID")) USING ("league_ID")', con=engine), indent = 4, sort_keys=True))

 \
 \
  \
  \
  \
  \
  \.
  

## tests

In [25]:
# soccer_data1 = {"Leagues": [{"league_ID": 1, "Name": "Bra1", "Link": "htmls....", "Country": "Brazil", "Tier": 1},\
#                             {"league_ID": 2, "Name": "Bra2", "Link": "htmls....", "Country": "Brazil", "Tier": 2}],\
#                "Clubs": [{"team_ID" : 1, "league_ID": 1, "club" : "Flamengo", "squad" : 30, "foreigners" : 2, "avg_market_m" : 14.5,
#                      "total_MV_m" : 50, "Logo_img" : "htmls....", "link_page" : "htmls...."},
#                         {"team_ID" : 1, "club" : "GOIAS", "squad" : 25, "foreigners" : 5, "avg_market_m" : 10.5,
#                      "total_MV_m" : 40, "Logo_img" : "htmls....", "link_page" : "htmls...."}],
#                "Players":[{"players_ID": 1, "name": "Diego", "position": "GoalKeeper","field_position": "GLK", "Age":39,"Nat": "Brazil", "Height": 1.81, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                          {"players_ID": 2, "name": "Marcos", "position": "Midfielder","field_position": "MID", "Age":25,"Nat": "USA", "Height": 1.82, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                          {"players_ID": 13, "name": "GoalKeeper_BR", "position": "GoalKeeper","field_position": "GLK", "Age":27,"Nat": "Brazil", "Height": 0, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                           {"players_ID": 3, "name": "Saulo", "position": "GoalKeeper","field_position": "GLK", "Age":27,"Nat": "Brazil", "Height": 1.83, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                           {"players_ID": 4, "name": "Marcos*2", "position": "Midfielder","field_position": "MID", "Age":25,"Nat": "Brazil", "Height": 1.84, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                          {"players_ID": 5, "name": "Saulo*2", "position": "GoalKeeper","field_position": "GLK", "Age":27,"Nat": "Brazil", "Height": 1.84, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                           {"players_ID": 6, "name": "Marcos**3", "position": "Midfielder","field_position": "MID", "Age":25,"Nat": "USA", "Height": 1.85, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                          {"players_ID": 7, "name": "Saulo**3", "position": "GoalKeeper","field_position": "GLK", "Age":27,"Nat": "Brazil", "Height": 1.86, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                           {"players_ID": 8, "name": "Marcos***4", "position": "Midfielder","field_position": "MID", "Age":25,"Nat": "USA", "Height": 1.87, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                          {"players_ID": 9, "name": "Saulo***4", "position": "GoalKeeper","field_position": "GLK", "Age":27,"Nat": "Brazil", "Height": 1.88, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                           {"players_ID": 10, "name": "Marcos****5", "position": "Midfielder","field_position": "MID", "Age":25,"Nat": "Brazil", "Height": 1.89, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                          {"players_ID": 11, "name": "Saulo****5", "position": "GoalKeeper","field_position": "GLK", "Age":27,"Nat": "USA", "Height": 1.90, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
#                           {"players_ID": 12, "name": "Midfielder_BR", "position": "Midfielder","field_position": "MID", "Age":25,"Nat": "Brazil", "Height": 0, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"},
                         
#                          {"players_ID": 14, "name": "MID_USA", "position": "Middd","field_position": "MID", "Age":29,"Nat": "USA", "Height": 0, "foot": "foot_1", "team_ID": 1, 'dt_joined': "dt_joined_1","prev_team": "prev_team_1","contract_expires": "contract_expires_1", "market_value": "market_value_1","player_page": "player_page"}
#                          ]}



In [26]:
# teams_df = "data/BRAZIL_1_BRA_A_trmk.csv"

# scrape_team_data1(teams_df)

In [27]:
# bra_1= pd.read_csv("data/teams_trmk.csv")
# bra_1

In [28]:
# bra_1.dtypes

In [29]:
# bra_1["club"] = bra_1["club"].astype(str)
# # bra_1.dtypes

In [30]:
# files = glob('data/*.csv', recursive=True)
# scrape_team_data1(files)


In [31]:
# df_league = pd.read_csv(teams_df)
# df_league

In [32]:
# contratcs = ["15.06.2024", "30.12.2024", "30.06.2024"]

# for contratc in contratcs:
# #     try:
# # #         contratc_expires_1 = datetime.strptime(contratc, '%d.%m.%Y').date()
# #         dt_joined_1 = datetime.strptime(contratc, '%b %d, %Y').date()
# #     except:
# # #         contract_expires_1 = "01/01/2030"
# #         dt_joined_1 = "2019-01-01"
#     try:
#         contratc_expires_1 = datetime.strptime(contratc, '%d.%m.%Y').date()
# #         dt_joined_1 = datetime.strptime(contratc, '%b %d, %Y').date()
#     except:
#         contract_expires_1 = "2030-01-01"
# #         dt_joined_1 = "01/01/2019"

#     print(contratc_expires_1)
    

In [33]:
# fla= pd.read_csv("data/players_trmk.csv")
# fla

In [34]:
# fla.dtypes

In [35]:
# flamengp = "https://www.transfermarkt.com/clube-de-regatas-do-flamengo/kader/verein/614/saison_id/2019/plus/1"
# team_name = goias.split('/')[-8]
# print(f"scraping: {team_name}")
# team_id = 1
      

    
# html = requests.get(goias, headers=headers)
# soup = BeautifulSoup(html.content, 'html.parser')
# htmltable = soup.find('table', class_= "items")

# results = htmltable.findAll("tr", class_ =["odd","even"])

# player_id = 1
# name = []
# player_page = []
# position = []
# f_posi = []
# Age = []
# Nat = []
# Height = []
# foot = []
# dt_joined = []
# prev_team = []
# contract_expires = []
# market_value = []
# team_ID = []
# players_ID = []


# for result in results:
#     features = result.findAll("td")

#     name_1 = features[2].find("img", alt=True)["alt"]

#     player_page.append("https://www.transfermarkt.com" + features[3].find("a", href=True)["href"])

#     position_1 = features[4].text
    
#     if position_1 in ("Centre-Back", "Left-Back", "Right-Back", "Defender"):
#         field_posit = "DEF"
#     elif position_1 in ("Defensive Midfield", "Central Midfield" , "Right Midfield" , "Left Midfield" , \
#     "Attacking Midfield" , "Midfield"):
#         field_posit = "MID"
#     elif position_1 in ("Left Winger" , "Right Winger" , "Centre-Forward" , "Second Striker" , "Forward"):
#         field_posit = "ATT"
#     else:
#         field_posit = "GLK"
#     try:
#         age_1 = int((features[5].text.split("(",)[-1])[:-1])
#     except:
#         age_1 = "-"

#     nat = features[6].img["alt"]

#     try:
#         Height_1 = float((features[7].text.split(" ")[0]).replace(",", "."))
#     except:
#         Height_1 = "0"

#     foot_1 = features[8].text

#     dt_joined_1 = features[9].text
#     try:
#         dt_joined_1 = datetime.strptime(dt_joined_1, '%b %d, %Y').date()
#     except:
#         dt_joined_1 = "2019-01-01"

#     try:
#         prev_team_1 = features[10].img["alt"]
#     except:
#         prev_team_1 = "-"


#     contract_expires_1 = features[11].text
#     try:
#         contratc_expires_1 = datetime.strptime(contratc_expires_1, '%d.%m.%Y').date()
#     except:
#         contract_expires_1 = "2030-01-01"


#     try:
#         market_value_1 = get_value_us(features[12].text[:-2])
#     except:
#         market_value_1 = 0

#     name.append(strip_accents(name_1))
#     position.append(position_1)
#     f_posi.append(field_posit)
#     Age.append(age_1)
#     Nat.append(nat)
#     Height.append(Height_1)
#     foot.append(foot_1)
#     dt_joined.append(dt_joined_1)
#     prev_team.append(strip_accents(prev_team_1))
#     contract_expires.append(contract_expires_1)
#     market_value.append(market_value_1)
#     team_ID.append(team_id)
#     players_ID.append(player_id)
#     player_id = player_id+1

# #     time.sleep(randint(3,5))

# df = pd.DataFrame(list(zip(players_ID, team_ID, name, position,f_posi, Age,Nat, Height, foot,dt_joined,prev_team, contract_expires,\
#                            market_value,player_page)), 
#                   columns =["players_ID", "team_ID","name","position","field_position", "Age", "Nat","Height","foot",'dt_joined',"prev_team", \
#                             "contract_expires", "market_value","player_page" ])
# df['players_ID'] = df['players_ID'].astype(int)
# df['team_ID'] = df['team_ID'].astype(int)
# df['dt_joined'] = pd.to_datetime(df['dt_joined'])
# df['contract_expires'] = pd.to_datetime(df['contract_expires'])


# df


In [36]:
# df.dtypes

In [37]:
# from datetime import datetime

# df_copy = df.copy()

# for date in df_copy["contract_expires"]:
#     try:
#         date = datetime.strptime(date, '%d.%m.%Y').date()
#     except:
#         date = '-' 
#     print(date)


In [38]:
# fla_copy = fla.copy()

# for date in fla_copy["dt_joined"]:
#     try:
#         date = datetime.strptime(date, '%b %d, %Y').date()
#     except:
#         date = "-"
#     print(date)

In [39]:
# name_1 = features[2].find("img", alt=True)["alt"]
# position_1 = features[4].text
# DoB_1 = int((features[5].text.split("(",)[-1])[:-1])
# nat = features[6].img["alt"]
# Height_1 = float((features[7].text.split(" ")[0]).replace(",", "."))
# foot_1 = features[8].text
# dt_joined_1 = features[9].text
# prev_team_1 = features[10].img["alt"]
# contract_expires_1 = features[11].text
# market_value_1 = get_value_us(features[12].text[:-2])

# print(name_1, position_1,DoB_1, nat, Height_1,foot_1,dt_joined_1,prev_team_1,contract_expires_1,market_value_1   )


In [40]:
# pd.read_csv("data/teams/goias-esporte-clube_trmk.csv")

In [41]:
# # Retrieve page with the requests module
# url = "https://www.transfermarkt.com/major-league-soccer/startseite/wettbewerb/MLS1"
# html = requests.get(url, headers=headers)

In [42]:
# # Create BeautifulSoup object; parse with 'html.parser'
# soup = BeautifulSoup(html.content, 'html.parser')
# htmltable = soup.find('table', class_= "items")
# text1 = htmltable.text
# text1

In [43]:
# #Examine the results, then determine element that contains sought info
# print(soup.prettify())

In [44]:
# results = htmltable.findAll("tr", class_ =["odd","even"])
# print(results[0])
# print(len(results))

In [45]:
# # to Download imgs
# for img in logos:
#         with open(basename(img), "wb") as f:
#             f.write(requests.get(img).content)

In [46]:
# names= []
# for result in results:
#     names.append(result.find("img", alt=True)["alt"])
    
# names

In [47]:
## Extrating a table from HTML
# htmltable2 = str(htmltable)
# dfs = pd.read_html(htmltable2)
# df_clean = dfs[0][["Club.1","name.1", "ø age", "Total market value", "ø market value" ]]
# df_clean = df_clean.drop(df_clean.index[len(df_clean.index)-1])
# df = df_clean.set_axis(["Club","Squad", "Foreigners", "avg_market_value" , "total_market_value"], axis=1, inplace=False)

<td class="zentriert no-border-rechts">
    <a class="vereinprofil_tooltip" href="/los-angeles-football-club/startseite/verein/51828/saison_id/2019">
        <img alt="Los Angeles FC" class="tiny_wappen" src="https://tmssl.akamaized.net//images/wappen/tiny/51828.png?lm=1511112738" title=" "/>
    </a>
</td>
<td class="hauptlink no-border-links hide-for-small hide-for-pad">
    <a class="vereinprofil_tooltip" href="/los-angeles-football-club/startseite/verein/51828/saison_id/2019" id="51828">
        Los Angeles FC
    </a>
</td>
<td class="hauptlink no-border-links show-for-small show-for-pad">
    <a class="vereinprofil_tooltip" href="/los-angeles-football-club/startseite/verein/51828/saison_id/2019" id="51828">
        Los Angeles FC
    </a>
</td>
<td class="zentriert">
    <a href="/los-angeles-fc/kader/verein/51828/saison_id/2019" title="Los Angeles FC">
    24
    </a>
</td>
<td class="zentriert hide-for-small hide-for-pad">
    26,1
</td>
<td class="zentriert hide-for-pad hide-for-small">
    17
</td>
<td class="rechts hide-for-small hide-for-pad">
    <a href="/los-angeles-fc/kader/verein/51828/saison_id/2019" title="Los Angeles FC">
    €49.20m
    </a>
</td>
<td class="rechts hide-for-small hide-for-pad">
    €2.05m
</td>
<td class="rechts show-for-small show-for-pad nowrap">
    <a href="/los-angeles-fc/kader/verein/51828/saison_id/2019" title="Los Angeles FC">
        €49.20m
    </a>
</td>
<td class="rechts show-for-small show-for-pad nowrap">
    €2.05m
</td>


In [48]:
# def get_value(x):
#     value = []
#     for char in x:
#         value.append(char)
#     float_value = "".join(value[1:-1])
#     return float(float_value)

In [49]:
# get_value(fdf[6].text)

In [50]:
# # URLs of page to be scraped
# bra = {"BRAZIL": {"leagues":[{"tier_1":'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1'},
#                  {"tier_2": "https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA2"}]}},
# us = {"UNITED_STATES": {"leagues":[{"tier_1": "https://www.transfermarkt.com/major-league-soccer/startseite/wettbewerb/MLS1"},
#                   {"tier_2":"https://www.transfermarkt.com/usl-pro/startseite/wettbewerb/USL"}, 
#                   {"tier_3":"https://www.transfermarkt.com/usl-league-one/startseite/wettbewerb/USC3"}]}}
# urls= [bra, us]

In [51]:
# def scrape_league_data(leagues_df):
#     team_id = 1
#     for index, row in leagues_df.iterrows():
#         url = row["league_link"]
#         league_name = row["league_name"]
#         tier = row["tier"]
#         country = row["country"]
#         league_id = row["league_ID"]
#         print(f"scraping: {country}_{tier}_{league_name}")
#         html = requests.get(url, headers=headers)
#         soup = BeautifulSoup(html.content, 'html.parser')
#         htmltable = soup.find('table', class_= "items")


#         results = htmltable.findAll("tr", class_ =["odd","even"])


#         team_ID = []
#         links = []
#         names = []
#         logos = []
#         squads = []
#         foreigners = []
#         total_MVs = []
#         avg_MVs = []
#         league_ID = []



#         for result in results:
#             features = result.findAll("td")
#             links.append(("https://www.transfermarkt.com"+result.find("a", href=True)\
#                           ["href"]+"/plus/1").replace("startseite", "kader"))
#             logo = result.find("img", src=True)["src"]
#             logo = logo.split("?")[0]
#             logo = logo.replace("tiny", "header")
#             logos.append(logo)
#             name = features[1].text
#             names.append(name)
#             squad = features[3].text
#             squads.append(squad)
#             foreigner = features[5].text
#             foreigners.append(foreigner)
#             total_MV = get_value_us(features[6].text)
#             total_MVs.append(total_MV)
#             avg_MV = get_value_us(features[7].text)
#             avg_MVs.append(avg_MV)
#             team_ID.append(team_id)
#             league_ID.append(league_id)
#             team_id = team_id + 1



#         # Create a Dataframe and export to a .csv file
#         df = pd.DataFrame(list(zip(team_ID, league_ID, names, squads, foreigners,avg_MVs, total_MVs, logos,links)), \
#     columns =["team_ID", "league_ID","club","squad", "foreigners", "avg_market_value_m", "total_MV_m",'Logo_img', "link_page"]) 
#         df['league_ID'] = df['league_ID'].astype(int)
#         df['team_ID'] = df['team_ID'].astype(int)

#         df.to_csv(f'data/{country}_{tier}_{league_name}_trmk.csv',index=False)


#         time.sleep(1)

In [52]:
# def scrape_team_data(files):
#     player_id = 1
#     for filename in files:
#         print(filename)
#         time.sleep(5)
#         df_league = pd.read_csv(filename)
#         for index, row in df_league.iterrows():
#             team_id = row["team_ID"]
#             team = row["link_page"]
#             team_name = team.split('/')[-8]
#             print(team_name)
#             html = requests.get(team, headers=headers)
#             soup = BeautifulSoup(html.content, 'html.parser')
#             htmltable = soup.find('table', class_= "items")

#             results = htmltable.findAll("tr", class_ =["odd","even"])
#             name = []
#             player_page = []
#             position = []
#             Age = []
#             Nat = []
#             Height = []
#             foot = []
#             dt_joined = []
#             prev_team = []
#             contract_expires = []
#             market_value = []
#             team_ID = []
#             players_ID = []

#             for result in results:
#                 features = result.findAll("td")

#                 name_1 = features[2].find("img", alt=True)["alt"]

#                 player_page.append("https://www.transfermarkt.com" + features[3].find("a", href=True)["href"])

#                 position_1 = features[4].text

#                 try:
#                     age_1 = int((features[5].text.split("(",)[-1])[:-1])
#                 except:
#                     age_1 = "-"

#                 nat = features[6].img["alt"]

#                 try:
#                     Height_1 = float((features[7].text.split(" ")[0]).replace(",", "."))
#                 except:
#                     Height_1 = "m"

#                 foot_1 = features[8].text

#                 dt_joined_1 = features[9].text
#                 try:
#                     dt_joined_1 = datetime.strptime(dt_joined_1, '%b %d, %Y').date()
#                 except:
#                     dt_joined_1 = "-"

#                 try:
#                     prev_team_1 = features[10].img["alt"]
#                 except:
#                     prev_team_1 = "-"


#                 contract_expires_1 = features[11].text
#                 try:
#                     contratc_expires_1 = datetime.strptime(contratc_expires_1, '%d.%m.%Y').date()
#                 except:
#                     contract_expires_1 = features[11].text


#                 try:
#                     market_value_1 = get_value_us(features[12].text[:-2])
#                 except:
#                     market_value_1 = 0

#                 name.append(name_1)
#                 position.append(position_1)
#                 Age.append(age_1)
#                 Nat.append(nat)
#                 Height.append(Height_1)
#                 foot.append(foot_1)
#                 dt_joined.append(dt_joined_1)
#                 prev_team.append(prev_team_1)
#                 contract_expires.append(contract_expires_1)
#                 market_value.append(market_value_1)
#                 team_ID.append(team_id)
#                 players_ID.append(player_id)
#                 player_id = player_id+1

#             df = pd.DataFrame(list(zip(players_ID, team_ID, name, position, Age,Nat, Height, foot,dt_joined,prev_team, contract_expires,\
#                                        market_value,player_page)), 
#                               columns =["players_ID", "team_ID","name","position", "Age", "Nat","Height","foot",'dt_joined',"prev_team", \
#                                         "contract_expires", "market_value","player_page" ])
#             df['players_ID'] = df['players_ID'].astype(int)
#             df['team_ID'] = df['team_ID'].astype(int)
            

#             df.to_csv(f"data/teams/{team_name}_trmk.csv" ,index=False)
#             time.sleep(1)